In [1]:
import torch
import os
import torch.nn as nn
from vae import VAE
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.optim import Adam
from torchvision.datasets import ImageFolder

data_path = '../data/train_data_images'
# train_dataset = ImageFolder(
#     root=data_path,
#     transform=transforms.ToTensor()
# )

# train_loader = DataLoader(
#     train_dataset,
#     batch_size=64,
#     num_workers=1,
#     shuffle=True
# )

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = VAE().to(device)
optimizer = Adam(model.parameters(), lr=1e-3)


def loss_function(x, x_hat, mean, log_var):
    reproduction_loss = nn.functional.binary_cross_entropy(x_hat, x, reduction='sum')
    KLD = - 0.5 * torch.sum(1+ log_var - mean.pow(2) - log_var.exp())

    return reproduction_loss + KLD

batch_size = 100
#train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)


#train(model, optimizer, epochs=50, device=device)

In [7]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        
        # Get all image paths
        for img_file in os.listdir(root_dir):
            if img_file.endswith(".png"):  # Adjust for other formats if needed
                self.image_paths.append(os.path.join(root_dir, img_file))

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")  # Ensure 3-channel images

        if self.transform:
            image = self.transform(image)

        return image  # If labels are needed, return (image, label)

In [19]:
transform = transforms.Compose([
#    transforms.Resize((30, 30)),  
    transforms.ToTensor(),
#    transforms.Normalize((0.5,), (0.5,))
])

# Define dataset and dataloaders
train_dataset = CustomDataset(root_dir=data_path, transform=transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
#test_dataset = CustomDataset(root_dir=data_path, transform=transform)


def train(model, optimizer, epochs, device, x_dim=784):
    model.train()
    for epoch in range(epochs):
        overall_loss = 0
        for batch_idx, (x, _) in enumerate(train_loader):
            x = x.view(batch_size, x_dim).to(device)

            optimizer.zero_grad()

            x_hat, mean, log_var = model(x)
            loss = loss_function(x, x_hat, mean, log_var)
            
            overall_loss += loss.item()
            
            loss.backward()
            optimizer.step()

        print("\tEpoch", epoch + 1, "\tAverage Loss: ", overall_loss/(batch_idx*batch_size))
    return overall_loss


In [20]:
train(model, optimizer, epochs=50, device=device)

RuntimeError: stack expects each tensor to be equal size, but got [3, 7, 23] at entry 0 and [3, 24, 8] at entry 1